# Extracting sentences using MatBERT

In [ ]:
from utils import *

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from torch.utils.data import DataLoader
from datasets import Dataset

# import warnings
# from torch.optim import AdamW
# from transformers import get_scheduler
# from transformers.utils import logging
# import evaluate
# from torch.utils.data import WeightedRandomSampler

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

First we load the data. Here, `corpus_all_paragraphs` should be a dictionnary with the dois as keys. Each paper is thus separated. For each entry, the dictionnary should contain a list of paragraphs in the form of dicionnaries with, at least, a 'text' entry.

Schematically, the data should looks like this:

{

doi1: [{'text': "Some text"}, {'text': "Some text"}, {'text': "Some text"}],
    
doi2: [...],
    
...

}

In [ ]:
# Load data
with open("../data_by_corpus/from_papers_v2/all_paragraphs.json", 'r') as f:
    corpus_all_paragraphs = json.load(f)

In [ ]:
# Load tokenizer and MatBERT
tokenizer = AutoTokenizer.from_pretrained("matbert-base-uncased/")
model = AutoModelForSequenceClassification.from_pretrained("matbert-sentence-extraction/").to(device)

# Create custom sentence tokenizer with abbreviations
punkt_param = PunktParameters()
abbreviation = ['fig', 'al', 'e.g', 'i.e']
punkt_param.abbrev_types = set(abbreviation)
sentence_tokenizer = PunktSentenceTokenizer(punkt_param)

Here we loop over every paper and paragraph and evaluate our MatBERT model on each sentence as tokenized by `sentence_tokenizer`. If the sentence's logit prediction is 1, then we add that to `extracted_sentences`. The latter is a list of dictionnaries with each dict containing the keys `sentences`, `doi`, and `paragraph`. `sentences` is a list of sentences extracted from `paragraph` of the paper with doi `doi`.

In [ ]:
extracted_sentences = []

all_dois = list(corpus_all_paragraphs.keys())

for i, doi in enumerate(all_dois):
    for para in corpus_all_paragraphs[doi]:

        # To prevent extracting the same paragraph twice
        if len(extracted_sentences)>0:
            if para['_id'] in [s['paragraph']['_id'] for s in extracted_sentences]:
                continue

        # Some paragraphs could be empty
        if len(para['text'])==0:
            continue

        # Get all the sentences in the paper
        sentences = sentence_tokenizer.tokenize(para['text'])

        # Prepare sentences to pass in model (tokenize them)
        sent_tok = tokenizer(sentences, padding=True, truncation=True, max_length=512)
        sent_tok['sentences'] = sentences
        test_data = Dataset.from_dict(sent_tok)
        test_data.set_format('torch')
        test_dataloader = DataLoader(test_data, batch_size=8)

        # Evaluate model
        predictions = []
        model.eval()
        for batch in test_dataloader:
            batch_sentences = batch['sentences']
            batch = {k: v.to(device) for k, v in batch.items() if torch.is_tensor(v)}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().tolist())

        # If we have at least one good sentence, add it/them to list
        if sum(predictions)>0:
            to_add = {}
            to_add['sentences'] = np.array(sentences)[np.array(predictions)==1].tolist()
            to_add['doi'] = doi
            to_add['paragraph'] = para
            extracted_sentences.append(to_add)
            break

In [3]:
from utils import *

# Training best BERT model
The manually labeled data used for hyperparameter tuning of various BERT models can be found in "data/manually_labelled_sentences.csv". Each sentence has a label of either 1 for extraction or 0 for non extraction.£

In [7]:
pd.read_csv("data/manually_labelled_sentences.csv", index_col=0)

,sentence,extract
0,Cr is repelled strongly yielding a Cr enriched...,0
1,This result suggests that the local chemical b...,0
2,"However, a high fraction of δ′ phase is not fa...",1
3,Low melting point T- and S-phases affect hot w...,1
4,The so-called Weldalite series of Al–Cu–Li–Mg–...,1
...,...,...
2046,The present study involves the experimental ev...,1
2047,Table 2 shows that the corrosion current densi...,0
2048,This is attributed to the fact that the η'-pha...,1
2049,Mathieu et al.51 suggested that the slower cor...,0
